In [1]:
!pip install kneed

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

import re
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime ,timedelta

from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

In [6]:
data = pd.read_csv('/content/drive/MyDrive/KPMG/clean_data.csv',index_col=0) 

In [7]:
data.head(2)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,lasted_for_years,discount_amt,gender,past_3_years_bike_related_purchases,job_industry_category,wealth_segment,owns_car,tenure,state,property_valuation,age
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,5,17.87,Male,19,Financial Services,Mass Customer,Yes,10.0,Victoria,6,62.0
1,11065,1,2950,2017-10-16,0.0,Approved,Giant Bicycles,Standard,medium,medium,1403.50,954.82,14,448.68,Male,19,Financial Services,Mass Customer,Yes,10.0,Victoria,6,62.0


In [ ]:
data['month'] = data.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d').month)

In [ ]:
data.head(1)
# past_3_years_bike_related_purchases as frequency, standard_cost as monetary, month can be used a recency [check if the months are 1-12? as only for year 2017]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,lasted_for_years,discount_amt,gender,past_3_years_bike_related_purchases,job_industry_category,wealth_segment,owns_car,tenure,state,property_valuation,age,month
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,5,17.87,Male,19,Financial Services,Mass Customer,Yes,10.0,Victoria,6,62.0,2


In [ ]:
# calculating recency(R), frequency(F), monetary(M) for the data.

rfm = data.groupby(['customer_id']).agg(recency=('month',np.max), # used max month to get the latest month the person made transaction
                                  frequency = ('past_3_years_bike_related_purchases',np.median), # used past purchases to get the frequency
                                  monetary = ('standard_cost',np.sum)) # for monetary i used the total sales of the person 

In [ ]:
rfm.head()

,recency,frequency,monetary
customer_id,,,
1,12,93,6066.36
2,8,81,1922.81
4,6,33,827.15
5,12,56,3508.26
6,10,35,1985.14


In [ ]:
# using quartiles to cut the data into four parts

rfm['r_quartile'] = pd.qcut(rfm['recency'], 4, ['3','2','1'], duplicates='drop') # this is only cut into 3 bins
rfm['f_quartile'] = pd.qcut(rfm['frequency'], 4, ['4','3','2','1'])
rfm['m_quartile'] = pd.qcut(rfm['monetary'], 4, ['4','3','2','1'])

In [ ]:
# getting the total score. 

rfm['RFM_Score'] = rfm.r_quartile.astype(str)+ rfm.f_quartile.astype(str) + rfm.m_quartile.astype(str)
rfm.head()

,recency,frequency,monetary,f_quartile,m_quartile,r_quartile,RFM_Score
customer_id,,,,,,,
1,12,93,6066.36,1,1,1,111
2,8,81,1922.81,1,4,3,314
4,6,33,827.15,3,4,3,334
5,12,56,3508.26,2,2,1,122
6,10,35,1985.14,3,3,3,333


There will be 48(3x4x4) sets were created. 111 being the best customers, and 344 being the least profitable. 

For getting the potential customer from the new customers we will have to focus on the characteristics of the group of customer in the 111 score.

In [ ]:
# Filter out Top/Best cusotmers

rfm[rfm['RFM_Score']=='111'].sort_values('monetary', ascending=False).head()

,recency,frequency,monetary,f_quartile,m_quartile,r_quartile,RFM_Score
customer_id,,,,,,,
1597,12,93,8313.22,1,1,1,111
1517,12,90,7871.93,1,1,1,111
2816,12,87,7684.26,1,1,1,111
2836,12,80,7618.71,1,1,1,111
173,12,99,7511.97,1,1,1,111


In [ ]:
customer_id = rfm[rfm['RFM_Score'] == '344'].index # choose any value for the rfm_score

In [ ]:
# filtering customers with only the provided rfm_score
demographic.set_index('customer_id', inplace=True) 

In [ ]:
demographic.head(1)

,gender,past_3_years_bike_related_purchases,job_industry_category,wealth_segment,owns_car,tenure,state,property_valuation,age
customer_id,,,,,,,,,
1,Female,93,Health,Mass Customer,Yes,11.0,New South Wales,10,64.0


In [ ]:
# getting the median age
demographic.iloc[customer_id,:]['age'].median() 

40.0

In [ ]:
# getting the gender distribution
demographic.iloc[customer_id,:]['gender'].value_counts()/len(customer_id) *100

Female     54.310345
Male       43.965517
Unknown     1.724138
Name: gender, dtype: float64

In [ ]:
# getting the job category distribution
demographic.iloc[customer_id,:]['job_industry_category'].value_counts()/len(customer_id) *100

Manufacturing         21.551724
Financial Services    18.103448
Unknown               15.517241
Health                12.931034
Retail                12.068966
Argiculture            5.172414
IT                     5.172414
Property               3.448276
Entertainment          3.448276
Telecommunications     2.586207
Name: job_industry_category, dtype: float64

In [ ]:
# getting the wealth segemnt distribution
demographic.iloc[customer_id,:]['wealth_segment'].value_counts()/len(customer_id) *100

Mass Customer        53.448276
High Net Worth       23.275862
Affluent Customer    23.275862
Name: wealth_segment, dtype: float64

In [ ]:
# getting the state distribution
demographic.iloc[customer_id,:]['state'].value_counts()/len(customer_id) *100

New South Wales    52.586207
Victoria           26.724138
Queensland         20.689655
Name: state, dtype: float64

by doing the RFM analysis we will be getting 48 groups.

For 111 (best customers)
- AGE(median) = 40
- gender = 53% M, 40% F
- job category = 21% to manufacturing
- wealth segment = 52% mass, 24% high, 23% aff
- state - 57% NWS, 29% vic, 13% - queens

For 344 (least profitable)
- AGE(median) = 40
- gender = 53% M, 40% F
- job category = 21% to manufacturing
- wealth segment = 52% mass, 24% high, 23% aff
- state - 57% NWS, 29% vic, 13% - queens

By analysing the two groups we can see that there is not much difference in the two groups. So it is difficult to get potential customer from this analysis